Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.316860
Training accuracy: 6.8%
Validation accuracy: 9.3%
Loss at step 100: 2.320034
Training accuracy: 71.1%
Validation accuracy: 70.5%
Loss at step 200: 1.864014
Training accuracy: 74.3%
Validation accuracy: 73.1%
Loss at step 300: 1.619954
Training accuracy: 75.3%
Validation accuracy: 73.8%
Loss at step 400: 1.458816
Training accuracy: 76.2%
Validation accuracy: 74.3%
Loss at step 500: 1.340656
Training accuracy: 76.8%
Validation accuracy: 74.3%
Loss at step 600: 1.248235
Training accuracy: 77.5%
Validation accuracy: 74.6%
Loss at step 700: 1.172727
Training accuracy: 78.0%
Validation accuracy: 74.7%
Loss at step 800: 1.109341
Training accuracy: 78.4%
Validation accuracy: 74.8%
Test accuracy: 83.0%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.178885
Minibatch accuracy: 9.4%
Validation accuracy: 8.8%
Minibatch loss at step 500: 2.142731
Minibatch accuracy: 71.1%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 0.747642
Minibatch accuracy: 82.8%
Validation accuracy: 76.1%
Minibatch loss at step 1500: 0.955326
Minibatch accuracy: 77.3%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 1.041914
Minibatch accuracy: 76.6%
Validation accuracy: 77.0%
Minibatch loss at step 2500: 1.069952
Minibatch accuracy: 75.0%
Validation accuracy: 77.6%
Minibatch loss at step 3000: 0.820180
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Test accuracy: 86.7%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [8]:
batch_size = 128
num_steps = 30000 + 1

#input data and labels
x = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
y_true = tf.placeholder(tf.float32, shape=(None, num_labels))

#hidden layer
W_1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
b_1 = tf.Variable(tf.zeros([1024]))
h_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)

#output layer
W_2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
b_2 = tf.Variable(tf.zeros([num_labels]))
logits = tf.matmul(h_1, W_2) + b_2
prob = tf.nn.softmax(logits)

#loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y_true))

#optimizer.
optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

#accuracy
correct_prediction = tf.equal(tf.argmax(prob, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        _, l, a = sess.run([optimizer, loss, accuracy], feed_dict={x : batch_data, y_true : batch_labels})
        if step % 2000 == 0:
            print("step %d" % step)
            print("minibatch loss %f" % l)
            print("minibatch accuracy %f" % a)
            a = sess.run(accuracy, feed_dict={x: valid_dataset, y_true: valid_labels})
            print("validation accuracy %f" % a)
        
    print(sess.run(accuracy, feed_dict={x: test_dataset, y_true: test_labels}))

step 0
minibatch loss 414.924866
minibatch accuracy 0.109375
validation accuracy 0.266300
step 2000
minibatch loss 10.113770
minibatch accuracy 0.812500
validation accuracy 0.812000
step 4000
minibatch loss 7.987518
minibatch accuracy 0.804688
validation accuracy 0.830600
step 6000
minibatch loss 8.075667
minibatch accuracy 0.789062
validation accuracy 0.812800
step 8000
minibatch loss 3.652697
minibatch accuracy 0.851562
validation accuracy 0.843600
step 10000
minibatch loss 2.564917
minibatch accuracy 0.882812
validation accuracy 0.849300
step 12000
minibatch loss 0.875300
minibatch accuracy 0.929688
validation accuracy 0.841400
step 14000
minibatch loss 4.826624
minibatch accuracy 0.828125
validation accuracy 0.844800
step 16000
minibatch loss 1.405133
minibatch accuracy 0.914062
validation accuracy 0.850300
step 18000
minibatch loss 0.727525
minibatch accuracy 0.929688
validation accuracy 0.846400
step 20000
minibatch loss 3.688664
minibatch accuracy 0.875000
validation accuracy 0.

In [9]:
print(train_dataset.shape)
print(valid_dataset.shape)
print(test_dataset.shape)

train_dataset.flags.writeable = False
valid_dataset.flags.writeable = False
test_dataset.flags.writeable = False

from itertools import izip

data_label = dict()

def add_dataset(dataset, labels):
    for data, label in izip(dataset, labels):
        if data.data not in data_label:
            data_label[data.data] = set()
        data_label[data.data].add(label)
        
add_dataset(train_dataset, save["train_labels"])
add_dataset(valid_dataset, save["valid_labels"])
add_dataset(test_dataset, save["test_labels"])

ambiguous_data = set(data for data, label_set in data_label.iteritems() if len(label_set) > 1)

occurred = set()
# remove ambiguous and deduplicate
def cleaned(dataset, labels):
    mask = []
    for data in dataset:
        mask.append(data.data not in ambiguous_data and data.data not in occurred)
        occurred.add(data.data)
    mask = np.array(mask)
    return dataset[mask, :], labels[mask, :]

train_dataset_cleaned, train_labels_cleaned = cleaned(train_dataset, train_labels)
valid_dataset_cleaned, valid_labels_cleaned = cleaned(valid_dataset, valid_labels)
test_dataset_cleaned, test_labels_cleaned = cleaned(test_dataset, test_labels)

print(train_labels_cleaned.shape)
print(valid_labels_cleaned.shape)
print(test_labels_cleaned.shape)

(200000, 784)
(10000, 784)
(10000, 784)
(187238, 10)
(8926, 10)
(8590, 10)


In [12]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_cleaned.shape[0] - batch_size)
        batch_data = train_dataset_cleaned[offset:(offset + batch_size), :]
        batch_labels = train_labels_cleaned[offset:(offset + batch_size), :]
        _, l, a = sess.run([optimizer, loss, accuracy], feed_dict={x : batch_data, y_true : batch_labels})
        if step % 2000 == 0:
            print("step %d" % step)
            print("minibatch loss %f" % l)
            print("minibatch accuracy %f" % a)
            a = sess.run(accuracy, feed_dict={x: valid_dataset_cleaned, y_true: valid_labels_cleaned})
            print("validation accuracy %f" % a)
        
    print("accuracy on test set:", 
          sess.run(accuracy, feed_dict={x: test_dataset_cleaned, y_true: test_labels_cleaned}))    
    print("accuracy on whole train set", 
          sess.run(accuracy, feed_dict={x: train_dataset_cleaned, y_true: train_labels_cleaned}))

step 0
minibatch loss 435.737854
minibatch accuracy 0.062500
validation accuracy 0.239301
step 2000
minibatch loss 8.672884
minibatch accuracy 0.835938
validation accuracy 0.820636
step 4000
minibatch loss 2.985936
minibatch accuracy 0.890625
validation accuracy 0.832512
step 6000
minibatch loss 2.847493
minibatch accuracy 0.875000
validation accuracy 0.833744
step 8000
minibatch loss 3.322642
minibatch accuracy 0.835938
validation accuracy 0.826462
step 10000
minibatch loss 3.602205
minibatch accuracy 0.851562
validation accuracy 0.832400
step 12000
minibatch loss 2.639723
minibatch accuracy 0.859375
validation accuracy 0.834528
step 14000
minibatch loss 1.465587
minibatch accuracy 0.914062
validation accuracy 0.838337
step 16000
minibatch loss 1.776598
minibatch accuracy 0.851562
validation accuracy 0.831055
step 18000
minibatch loss 1.596829
minibatch accuracy 0.875000
validation accuracy 0.840914
step 20000
minibatch loss 1.546901
minibatch accuracy 0.914062
validation accuracy 0.8